### Study of Peter Norvig's *pytudes*

In [15]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, product, chain
from functools   import lru_cache, reduce
from typing      import Dict, Tuple, Set, List, Iterator, Optional, Union, Sequence
from contextlib  import contextmanager

import operator
import math
import ast
import sys
import re
import os

In [24]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    with open(os.path.join('inputs',str(day) + '.txt')) as f:
        sections = f.read().rstrip().split(sep)
        return list(map(parser, sections))
     
def do(day, *answers) -> List[int]:
    "E.g., do(3) returns [day3_1(in3), day3_2(in3)]. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part: 
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
        else:
            got.append(None)
    return got

In [8]:
Number = Union[float, int]
Atom = Union[Number, str]
Char = str # Type used to indicate a single character

cat = ''.join
flatten = chain.from_iterable

def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)

def prod(numbers) -> Number:
    "The product of an iterable of numbers." 
    return reduce(operator.mul, numbers, 1)

def dot(A, B) -> Number: 
    "The dot product of two vectors of numbers."
    return sum(a * b for a, b in zip(A, B))

def ints(text: str) -> Tuple[int]:
    "Return a tuple of all the integers in text."
    return mapt(int, re.findall('-?[0-9]+', text))

def lines(text: str) -> List[str]:
    "Split the text into a list of lines."
    return text.strip().splitlines()

def mapt(fn, *args): 
    "Do map(fn, *args) and make the result a tuple."
    return tuple(map(fn, *args))

def atoms(text: str, ignore=r'', sep=None) -> Tuple[Union[int, str]]:
    "Parse text into atoms separated by sep, with regex ignored."
    text = re.sub(ignore, '', text)
    return mapt(atom, text.split(sep))

def atom(text: str, types=(int, str)):
    "Parse text into one of the given types."
    for typ in types:
        try:
            return typ(text)
        except ValueError:
            pass

@contextmanager
def binding(**kwds):
    "Bind global variables within a context; revert to old values on exit."
    old_values = {k: globals()[k] for k in kwds}
    try:
        globals().update(kwds)
        yield # Stuff within the context gets run here.
    finally:
        globals().update(old_values)

In [25]:
data = data(1,int)


[1411, 1802, 1773, 1775, 1442, 1471, 1048, 1403, 1881, 1930, 1710, 1717, 685, 1255, 1451, 1870, 208, 1725, 1879, 143, 1372, 1726, 1357, 1624, 1378, 1993, 1721, 1712, 1867, 1355, 1743, 1942, 114, 407, 1892, 1937, 2001, 1466, 1461, 1770, 1441, 1410, 1915, 1482, 1512, 1631, 1954, 1632, 1788, 1971, 1989, 1427, 1684, 1749, 1795, 1839, 1358, 1354, 1591, 1924, 1456, 2002, 1746, 1323, 1946, 1889, 296, 1908, 1959, 1944, 1655, 1602, 1768, 1666, 1465, 1782, 1739, 1472, 1576, 645, 1496, 1538, 1761, 1353, 1639, 1904, 1765, 1519, 1948, 1900, 1376, 1918, 1950, 667, 1976, 1925, 1939, 1319, 1895, 1510, 1480, 735, 1674, 1997, 1868, 1728, 1893, 1500, 1363, 1840, 1905, 1361, 1894, 1558, 1369, 1922, 1367, 1463, 1365, 1504, 1898, 1343, 1436, 1700, 1911, 1811, 1829, 1984, 1444, 1806, 1455, 1778, 1835, 1817, 1668, 1907, 1748, 2007, 1534, 1269, 1473, 1572, 2006, 1651, 1853, 1943, 1968, 1969, 1437, 1692, 1955, 1964, 1821, 1805, 1999, 1614, 1754, 1888, 1832, 1623, 1723, 1678, 2008, 1819, 1595, 1972, 1229, 1703, 